# D2E2S Phase 1 Training — All Datasets

Training notebook for the DESS (D2E2S) ABSA model with Phase 1 improvements:
- Differential learning rates (transformer 1e-5, task heads 3e-4)
- Reduced batch_loss weight (2.0 instead of 10.0)
- Focal loss for sentiment classification
- Cosine annealing LR schedule

Target: Kaggle P100 (16GB) with `deberta-v3-base` and batch_size 16.

## 1. Environment Setup

In [ ]:
import os
print(os.getcwd())
os.chdir('/kaggle/working')
print('Working directory:', os.getcwd())

In [ ]:
!pip install -r requirements.txt

In [ ]:
import torch
print(f'PyTorch version: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'GPU Memory: {torch.cuda.get_device_properties(0).total_mem / 1024**3:.1f} GB')

## 2. Download DeBERTa-v3-base Model

In [ ]:
from transformers import AutoModel, AutoTokenizer

model_name = 'microsoft/deberta-v3-base'
save_path = './deberta-v3-base'

if not os.path.exists(os.path.join(save_path, 'config.json')):
    print(f'Downloading {model_name}...')
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    tokenizer.save_pretrained(save_path)
    model.save_pretrained(save_path)
    del model, tokenizer
    torch.cuda.empty_cache()
    print('Download complete.')
else:
    print('DeBERTa model already downloaded.')

## 3. Training Configuration

Common flags for all datasets with Phase 1 improvements enabled.

In [ ]:
COMMON_FLAGS = (
    '--seed 42 '
    '--max_span_size 8 '
    '--batch_size 16 '
    '--epochs 120 '
    '--pretrained_deberta_name microsoft/deberta-v3-base '
    '--deberta_feature_dim 768 '
    '--hidden_dim 384 '
    '--emb_dim 768 '
    '--use_enhanced_semgcn '
    '--use_differential_lr '
    '--transformer_lr 1e-5 '
    '--task_lr 3e-4 '
    '--batch_loss_weight 2.0 '
    '--use_focal_loss '
    '--focal_gamma 2.0 '
    '--use_cosine_schedule'
)

DATASETS = ['14res', '14lap', '15res', '16res']

print('Phase 1 training flags:')
print(COMMON_FLAGS)

## 4. Train All Datasets

Each dataset trains for 120 epochs. Results are saved to `log/{dataset}/{timestamp}/`.

In [ ]:
# --- 14res (SemEval 2014 Restaurant) ---
!python train.py --dataset 14res {COMMON_FLAGS}

In [ ]:
# --- 14lap (SemEval 2014 Laptop) ---
!python train.py --dataset 14lap {COMMON_FLAGS}

In [ ]:
# --- 15res (SemEval 2015 Restaurant) ---
!python train.py --dataset 15res {COMMON_FLAGS}

In [ ]:
# --- 16res (SemEval 2016 Restaurant) ---
!python train.py --dataset 16res {COMMON_FLAGS}

## 5. Review Results

Check the latest result files for each dataset.

In [ ]:
import glob

for ds in DATASETS:
    print(f'\n{"="*60}')
    print(f'Dataset: {ds}')
    print(f'{"="*60}')
    result_dirs = sorted(glob.glob(f'log/{ds}/*/result/result8.txt'))
    if result_dirs:
        latest = result_dirs[-1]
        print(f'Latest result: {latest}')
        with open(latest) as f:
            print(f.read())
    else:
        print('No results found yet.')